# Deploy your model on a SageMaker Endpoint and predict

<div style="text-align:center">
    <img src="../media/manual.png" width="800"/>
</div>

## Setup environment

In [ ]:
import sagemaker
from sagemaker.model import Model
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.predictor import Predictor
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer

sagemaker_session = sagemaker.Session()
region = sagemaker.Session().boto_region_name
role = sagemaker.get_execution_role()
model_artefact = '<your-model-s3-path>'

## Create endpoint and predictor

In [ ]:
image_uri = sagemaker.image_uris.retrieve(framework='forecasting-deepar', region=region, version='1')

model = Model(
    model_data=model_artefact,
    image_uri=image_uri,
    name=name_from_base('forecasting-model'),
    role=role
)

model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

In [ ]:
predictor = Predictor(
    endpoint_name='<your-endpoint-name>',
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

## Make predictions and plot results

In [ ]:
import numpy as np
import pandas as pd
import sagemaker
import matplotlib.pyplot as plt

np.random.seed(1)
freq = 'H'
prediction_length = 48
context_length = 72
t0 = '2016-01-01 00:00:00'
data_length = 400
num_ts = 200
period = 24

time_series = []
for k in range(num_ts):
    level = 10 * np.random.rand()
    seas_amplitude = (0.1 + 0.3*np.random.rand()) * level
    sig = 0.05 * level # noise parameter (constant in time)
    time_ticks = np.array(range(data_length))
    source = level + seas_amplitude*np.sin(time_ticks*(2*np.pi)/period)
    noise = sig*np.random.randn(data_length)
    data = source + noise
    index = pd.date_range(start=t0, freq=freq, periods=data_length)
    time_series.append(pd.Series(data=data, index=index))

time_series_training = []
for ts in time_series:
    time_series_training.append(ts[:-prediction_length]) 

Now we can use the previously created `predictor` object. For simplicity, we will predict only the first few time series used for training, and compare the results with the actual data we kept in the test set.

In [ ]:
actuals = time_series[5]
ts_to_predict = time_series_training[5]

payload = {
    "instances": [
        {
            "start": "2016-01-01 00:00:00", 
            "target": list(ts_to_predict)
        }
    ],
    "configuration": {
        "output_types": ["quantiles"],
        "quantiles": ['0.1','0.5','0.9']
    }
}

In [ ]:
predictions = predictor.predict(payload)['predictions'][0]['quantiles']
p10 = predictions['0.1']
p50 = predictions['0.5']
p90 = predictions['0.9']

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(actuals[-prediction_length-context_length:], label='actuals')
plt.fill_between(actuals[-prediction_length:].index, p10, p90, color='y', alpha=0.5, label='80% confidence interval')
plt.plot(actuals[-prediction_length:].index, p50, label='prediction median', color = 'red')
plt.legend()
plt.show()